In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from collections import Counter
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, classification_report
from sklearn import tree
from statannot import add_stat_annotation
from itertools import combinations
from tqdm import tqdm
from sklearn.tree import _tree
from sklearn.decomposition import PCA

## DataSet

In [2]:
##Original Datasets
df = pd.read_csv()
columns = df.columns
columns = [i.replace(" ", "_") for i in columns]
df.columns = columns
df = df.loc[df["Diagnosis_simplified"].isin(["HB", "HCC", "NOS"])]
subset["Diagnosis_simplified"] = df["Diagnosis_simplified"]
variables = subset.columns.tolist()[:-1]
variables_bx = []
variables_SR = []

array_split = list(map(lambda x: x.split("_"), variables))

for i in range(len(array_split)):
    if "bx" in array_split[i]:
        variables_bx.append(variables[i])
    elif "SR" in array_split[i]:
        variables_SR.append(variables[i])
if "Diagnosis_simplifed" not in variables_bx:
    variables_bx.append("Diagnosis_simplified")
df_nano_bx = df[variables_bx].dropna()


### Data anonymization

In [4]:
# Create an empty dictionary to store the original and new column names
dict_genes = {}

# Iterate over the columns of the DataFrame
for i, col in enumerate(df_nano_bx.columns):
    # Create the new column name as "BSC_BM" followed by the index
    new_col = f"BSC_BM{i+1}"
    # Add the original and new column names to the dictionary
    dict_genes[col] = new_col
    # Rename the column in the DataFrame
    df_nano_bx.rename(columns={col: new_col}, inplace=True)

### Functions 

In [5]:
### Recursive feature elimination with cross validation

def RFE_fun(X_data, y_data, df_data):
    ## automatically select the number of features for RFE
    # create pipeline
    rfe = RFECV(estimator=DecisionTreeClassifier()) #RFECV automatically select the number of features 
    model = DecisionTreeClassifier()
    pipeline = Pipeline(steps=[('s',rfe),('m',model)])
    # evaluate model
    cv = LeaveOneOut()
    n_scores = cross_val_score(pipeline, X_data, y_data, scoring='f1_weighted', cv=cv, n_jobs=-1, error_score='raise')
    # report performance
    print('F1 score %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

    # Report which features were selected by RFE
    # define RFE
    rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
    # fit RFE
    rfe.fit(X_data, y_data)
    biomarkers = []
    biomarkers = df_data.columns.tolist()[:-1]
    dicc = {}
    list_of_dicts = []
    # summarize all features
    for i in range(X_data.shape[1]):
        #print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
        dicc = {"Biomarker": biomarkers[i], "Score": rfe.ranking_[i] }
        list_of_dicts.append(dicc)
    top_bk = pd.DataFrame.from_dict(list_of_dicts)   
    top_bk= top_bk.loc[top_bk["Score"] == 1]
    return(top_bk)

def Decision_tree_classifier_fun(X_data, y_data, name_output):
    
    name = "DT_"+name_output+".jpg"

    #Scale the data
    scaler = StandardScaler()
    # transform data
    X_fitted = scaler.fit_transform(X_data)
    # create loocv procedure
    cv = LeaveOneOut()
    #define model
    model = DecisionTreeClassifier(criterion='gini', max_depth=3)

    # enumerate splits
    y_true, y_pred = list(), list()
    F1_scores = []
    Precision_scores = []
    Recall_scores = []
    for train_ix, test_ix in cv.split(X):
        # split data
        X_train, X_test = X_fitted[train_ix, :], X_fitted[test_ix, :]
        y_train, y_test = y_data[train_ix], y_data[test_ix]
        # fit model
        model.fit(X_train, y_train)
        # evaluate model
        yhat = model.predict(X_test)
        y_true.append(y_test[0])
        y_pred.append(yhat[0])
        F1_scores.append(f1_score(y_true,y_pred,average='weighted'))
        Precision_scores.append(precision_score(y_true,y_pred,average='weighted', zero_division=0))
        Recall_scores.append(recall_score(y_true,y_pred,average='weighted', zero_division=0))

    print("F1 score:", mean(F1_scores), "Precsision score: ", mean(Precision_scores), "Recall score: ", mean(Recall_scores))
    
    
    ##Decisoin Tree
    plt.figure(figsize=(25,18))
    labels = list(Counter(y_data))
    tree.plot_tree(model.fit(X_data, y_data), feature_names=feature_names,
        class_names=labels, fontsize=22, filled=True, rounded=True)
    plt.savefig(name,bbox_inches = "tight")
    ## Confusion Matrix
    plt.figure(figsize=(10,9))
    name = "CM_"+name_output+".jpg"
    cf_matrix = confusion_matrix(y_true, y_pred)
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]

    group_percentages = ["{0:.2%}".format(value) for value in
                                 cf_matrix.flatten()/np.sum(cf_matrix)]


    label = [f"{v1}\n{v2}\n" for v1, v2 in
              zip(group_counts,group_percentages)]
    num = len(list(Counter(y_data)))

    label = np.asarray(label).reshape(num,num)
    ax = sns.heatmap(cf_matrix, annot=label, fmt='', cmap='Blues')
    sns.set(font_scale=1.7)
    classes = pd.Series(y).unique().tolist()
    classes.sort()
    #ax.set_title("Confusion Matrix")
    ax.set_xlabel('\nPredicted Category')
    ax.set_ylabel('Actual Category ');
    ax.xaxis.set_ticklabels(classes)
    ax.yaxis.set_ticklabels(classes)
    plt.savefig(name,bbox_inches = "tight")


def boxplot_fun(X_data, y_data, df, name_output):
    name = "Boxplots_"+name_output+".jpg"
    # create loocv procedure
    cv = LeaveOneOut()
    #define model
    model = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0, class_weight="balanced")
    # enumerate splits
    y_true, y_pred = list(), list()
    for train_ix, test_ix in cv.split(X):
        # split data
        X_train, X_test = X_data[train_ix, :], X_data[test_ix, :]
        y_train, y_test = y_data[train_ix], y_data[test_ix]
        # fit model
        model.fit(X_train, y_train)
    feature_importances = pd.DataFrame(model.feature_importances_,
                                index = top_bk["Biomarker"].head(10).tolist(), 
                                   columns =["Importance"]).sort_values("Importance", ascending=False)
    fig, axes = plt.subplots(2, 5, figsize=(20, 20))
    lista_prueba = [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4]]

    genes = feature_importances.head(10).index.tolist()

    for i,num in zip(genes, lista_prueba):
        a = df[["Diagnosis_simplified", i]]
        a = a.dropna()
        values = a["Diagnosis_simplified"].value_counts().to_string()
        values = values.replace("     ", ':') 
        values =  values.replace("\n", " " )
        label = "Diagnosis:  " +str(values)
        x_ordered = a["Diagnosis_simplified"].sort_values()


        ax = sns.boxplot(x=x_ordered,  y=i,  data=df, ax=axes[num[0], num[1]])
        #ax.set_xlabel(label)
        medians = a["Diagnosis_simplified"].value_counts()
        for xtick in ax.get_xticks():
            ax.text(xtick,medians[xtick],medians[xtick], 
                    horizontalalignment='center', verticalalignment="top", size='medium', in_layout=True, color='black',weight='semibold')
        add_stat_annotation(ax, data=df, x = "Diagnosis_simplified", y=i,
                            box_pairs=[("HB", "HCC"), ("HB", "NOS"),("HCC", "NOS")],
                            test='Mann-Whitney', text_format='star', loc='outside', verbose=2)

    plt.tight_layout()
    fig.savefig(name, bbox_inches='tight')
    return(feature_importances)


### Top Features

In [ ]:
### RFE runs 500 times ####
y = df_top_MLR["Diagnosis_simplified"]
X = df_top_MLR.drop("Diagnosis_simplified", axis=1)
feature_names = X.columns
X = np.array(X)
y = np.array(y)
y[(y == "NOS") | (y == "HCC")] = "Other"
print(Counter(y))

# Report which features were selected by RFE
# define RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
top_list = []
for i in range(500):
    # fit RFE
    rfe.fit(X, y)
    biomarkers = []
    biomarkers = df_top_MLR.columns.tolist()[:-1]
    dicc = {}
    list_of_dicts = []
    # summarize all features
    for i in range(X.shape[1]):
        #print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
        dicc = {"Biomarker": biomarkers[i], "Score": rfe.ranking_[i] }
        list_of_dicts.append(dicc)
    top_bk = pd.DataFrame.from_dict(list_of_dicts)   
    top_bk= top_bk.loc[top_bk["Score"] == 1]
    top_list.extend(top_bk["Biomarker"].tolist())
df_features = pd.DataFrame.from_dict(Counter(top_list), orient='index').reset_index()
df_features = df_features.sort_values([0], ascending= False).head(7)


### Decision Tree

In [ ]:

list_bk = df_features["index"].tolist()
list_bk.append("Diagnosis_simplified")
df_top = df_nano_bx[list_bk]
y = df_top["Diagnosis_simplified"]
X = df_top.drop("Diagnosis_simplified", axis=1)
feature_names = X.columns
feature_names = [dict_genes[x] for x in feature_names]
X = np.array(X)
y = np.array(y)
#Scale the data
print(Counter(y))
y[(y == "NOS") | (y == "HCC")] = "Other"


Decision_tree_classifier_fun(X,y, "df_features")

### Boxplots

In [ ]:
genes = [k for k, v in dict_genes.items() if v in list_BM_LR]

fig, axes = plt.subplots(3, 4, figsize=(25, 20))
lista_prueba = [[0, 0], [0, 1], [0, 2],[0,3], [1, 0], [1, 1], [1, 2],[1,3], [2, 0], [2,1], [2,2], [2,3]]

gene_label = [dict_genes[x] for x in genes]
for i,num, y_label in zip(genes, lista_prueba, gene_label):
    a = df[["Diagnosis_simplified", i]]
    a = a.dropna()
    values = a["Diagnosis_simplified"].value_counts().to_string()
    values = values.replace("     ", ':') 
    values =  values.replace("\n", " " )
    label = "Diagnosis:  " +str(values)
    x_ordered = a["Diagnosis_simplified"].sort_values()


    ax = sns.boxplot(x=x_ordered,  y=i,  data=df, ax=axes[num[0], num[1]])
    #ax.set_xlabel(label)  
    ax.set_ylabel(y_label)
    add_stat_annotation(ax, data=df, x = "Diagnosis_simplified", y=i,
                        box_pairs=[("HB", "HCC"), ("HB", "NOS"),("HCC", "NOS")],
                        test='Mann-Whitney', text_format='star', loc='outside', verbose=2)

plt.tight_layout()
fig.savefig( "boxplot.jpg" )
